# Рубежная контрольная работа

Эту работу выполнять не обязательно. Баллы, набранные за работу, просуммируются с баллами, которые были набраны за контрольную работу 1, и будут выставлены как оценка за промежуточную аттестацию "автоматом".

Сделайте копию ноутбука.

Присвойте переменной `v` значение - свой табельный номер ИСУ, это нужно для определения вашего варианта. Запустите код в следующей ячейке.

In [ ]:
from datetime import datetime
current_datetime = datetime.now()
print(current_datetime)
v = 333413 # по умолчанию вписан мой номер

2023-12-15 07:07:34.345586


# Общее задание

Нужно решить задачу бинарной классификации, предварительно построив признаковое описание объектов на основе нескольких таблиц.

Целевая переменная - пол клиента.

В качестве модели нужно использовать нейронную сеть, которую нужно строить с помощью `keras` или `torch` на выбор студента.

# Данные

Для всех последующих заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **transactions, tr_mcc_codes и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк.

Эти данные уже использовались в работе 2. Описание данных тут: https://drive.google.com/file/d/13sn8orjkoGfO1SVZYByXNmZQYFRfmOdp/view?usp=sharing

Скачать данные отсюда: https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L?usp=sharing

## Задание 1 (6 баллов)

В задании требуется на основе нескольких таблиц с данными сделать признаковое описание объектов.

Объектами являются клиенты. Клиенты идентифицируются с помощью `customer_id`, которые есть в таблицах **transactions** и **gender_train**. В качестве призаков нужно использовать даннее по категориям транзаций: ```mcc_code``` — mcc-код транзакции есть в таблицах **transactions** и **tr_mcc_codes**.


Для выбора варианта запустите код в следующей ячейке.



In [ ]:
print ('Список категорий')
feature_list = ['Одежда (также нужно искать "одежды")', 'Аптеки', 'Ценные бумаги', 'Магазины косметики и Салоны красоты', 'Образовательные услуги (а также школы, университеты)',
                'Алименты', 'Прокат автомобилей', 'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)', 'Видеоигры (искать надо по "видеоигр")',
                'Отели', 'Программирование', 'Спиртное и алкоголь', 'Предсказатели будущего', 'Азартные игры']
if (int(str(v)[4]) % 2) == 0:
  print (feature_list[::2])
else:
  print (feature_list[::-2])


if (int(str(v)[5]) % 2) == 0:
  print ('В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями')
else:
  print ('В качестве значений признаков используйте сумму по транзакциям с тратами и сумму по транзакциям с поступлениями')

Список категорий
['Азартные игры', 'Спиртное и алкоголь', 'Отели', 'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)', 'Алименты', 'Магазины косметики и Салоны красоты', 'Аптеки']
В качестве значений признаков используйте сумму по транзакциям с тратами и сумму по транзакциям с поступлениями


Каждая категория может быть связана с одним или несколькими mcc-кодами. Например, вот несколько mcc-кодов по категории "Одежда":
* 5621; Готовая женская одежда
* 5641; Детская одежда
* 5651; Одежда для всей семьи

Таким образом получится 14 признаков, то есть семь категорий, каждая из которых даст два значения: траты и поступления. Например, категория "Одежда" даёт признаки "Одежда траты" и "Одежда поступления".

Сделайте нормализацию значений признаков так, чтобы все означения менялись от 0 до 1.

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import drive
drive.mount('/content/drive')

tr_types = pd.read_csv("/content/drive/MyDrive/machine_lerning/Lab 2/Копия tr_types.csv", sep=";")
tr_mcc_codes = pd.read_csv("/content/drive/MyDrive/machine_lerning/Lab 2/Копия tr_mcc_codes.csv", on_bad_lines='skip', sep=";")
gender_train = pd.read_csv("/content/drive/MyDrive/machine_lerning/Lab 2/Копия gender_train.csv")
transactions = pd.read_csv("/content/drive/MyDrive/machine_lerning/Lab 2/Копия transactions.csv", nrows=1000000)

Mounted at /content/drive


In [ ]:
tr_mcc_codes

,mcc_code,mcc_description
0,742,Ветеринарные услуги
1,1711,"Генеральные подрядчики по вентиляции, теплосна..."
2,1731,Подрядчики по электричеству
3,1799,"Подрядчики, специализированная торговля — нигд..."
4,2741,Разнообразные издательства/печатное дело
...,...,...
179,9211,"Судовые выплаты, включая алименты и детскую по..."
180,9222,Штрафы
181,9311,Налоговые платежи
182,9399,"Правительственные услуги, нигде ранее не класс..."


In [ ]:
gender_train

,customer_id,gender
0,10928546,1
1,69348468,1
2,61009479,0
3,74045822,0
4,27979606,1
...,...,...
8395,90417572,0
8396,66837341,0
8397,10758984,1
8398,11376556,0


In [ ]:
transactions

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN
...,...,...,...,...,...,...
999995,98801178,269 10:29:06,5921,1010,-23016.59,127873
999996,98801178,270 10:26:45,5211,1010,-61380.88,10114209
999997,98801178,270 15:18:53,5921,1010,-11800.27,10105341
999998,98801178,271 12:47:13,5211,1010,-16170.59,10114209


In [ ]:
merged_data = pd.merge(transactions, tr_mcc_codes, on='mcc_code', how='left')
merged_data

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,mcc_description
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,"Звонки с использованием телефонов, считывающих..."
1,39026145,1 10:19:29,6011,7010,56147.89,NaN,Финансовые институты — снятие наличности автом...
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN,Денежные переводы
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN,"Различные продовольственные магазины — рынки, ..."
4,39026145,2 15:33:42,5499,1010,-920.83,NaN,"Различные продовольственные магазины — рынки, ..."
...,...,...,...,...,...,...,...
999995,98801178,269 10:29:06,5921,1010,-23016.59,127873,Магазины с продажей спиртных напитков на вынос...
999996,98801178,270 10:26:45,5211,1010,-61380.88,10114209,Лесо- и строительный материал
999997,98801178,270 15:18:53,5921,1010,-11800.27,10105341,Магазины с продажей спиртных напитков на вынос...
999998,98801178,271 12:47:13,5211,1010,-16170.59,10114209,Лесо- и строительный материал


In [ ]:
unique_mcc_codes = merged_data['mcc_description'].unique()
unique_mcc_codes

In [ ]:
categories = ['Азартные игры', 'Спиртное и алкоголь', 'Отели',
              'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)',
              'Алименты', 'Магазины косметики и Салоны красоты', 'Аптеки']

features = pd.DataFrame({'customer_id': gender_train['customer_id']})
for category in categories:
    mcc_codes = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains(category)]['mcc_code'].tolist()
    category_transactions = transactions[transactions['mcc_code'].isin(mcc_codes)]
    grouped_transactions = category_transactions.groupby(['customer_id', 'amount']).size().unstack(fill_value=0)

    features[f'{category} траты'] = grouped_transactions.apply(lambda row: row[row < 0].sum(), axis=1)
    features[f'{category} поступления'] = grouped_transactions.apply(lambda row: row[row > 0].sum(), axis=1)

features = features.fillna(0)

scaler = MinMaxScaler()
features_normalized = pd.DataFrame(scaler.fit_transform(features.iloc[:, 1:]),
                                   columns=features.columns[1:])

data = pd.merge(features[['customer_id']], gender_train, on='customer_id')
data = pd.concat([data, features_normalized], axis=1)

data

<ipython-input-38-d780f2572e78>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mcc_codes = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains(category)]['mcc_code'].tolist()


,customer_id,gender,Азартные игры траты,Азартные игры поступления,Спиртное и алкоголь траты,Спиртное и алкоголь поступления,Отели траты,Отели поступления,"Автозапчасти, Автошины и Станции техобслуживания (а также СТО) траты","Автозапчасти, Автошины и Станции техобслуживания (а также СТО) поступления",Алименты траты,Алименты поступления,Магазины косметики и Салоны красоты траты,Магазины косметики и Салоны красоты поступления,Аптеки траты,Аптеки поступления
0,10928546,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,69348468,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,61009479,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,74045822,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,27979606,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8395,90417572,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8396,66837341,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8397,10758984,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8398,11376556,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# PUT YOUR CODE HERE
scaler = MinMaxScaler()
data[features] = scaler.fit_transform(data[features])

## Вопросы к заданию 1

1. Для каждой категории выведите соответствующие ей mcc-коды с описаниями.
2. Выведите количество пустых ячеек по каждому признаку. По какому признаку отсутствует больше всего данных?
3. По какой категории самая большая разница между мужчинами и женщинами?

In [ ]:
# PUT YOUR CODE HERE

# Задание 2 (3 балла)

Постройте модель - многослойную нейронную сеть минимум с двумя скрытыми слоями и с dropout. Количество нейронов в каждом слое выберите самостоятельно.
Функции активации, метод оптимизации, скорость обучения, параметр для dropout - выберите самостоятельно.

Модель нужно построить с помощью keras или torch.

Настройте параметры модели.

In [ ]:
# PUT YOUR CODE HERE
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

model = Sequential()
model.add(Dense(128, input_dim=128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

opt = Adam(learning_rate=0.001)  # выбор оптимизатора и скорости обучения
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 128)               16512     
                                                                 
 dropout_19 (Dropout)        (None, 128)               0         
                                                                 
 dense_27 (Dense)            (None, 64)                8256      
                                                                 
 dropout_20 (Dropout)        (None, 64)                0         
                                                                 
 dense_28 (Dense)            (None, 1)                 65        
                                                                 
Total params: 24833 (97.00 KB)
Trainable params: 24833 (97.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

iris = datasets.load_iris()
X = iris.data
y = iris.target

y = to_categorical(y, num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/50
3/3 [==============================] - 1s 102ms/step - loss: 1.3619 - accuracy: 0.2500 - val_loss: 1.0844 - val_accuracy: 0.5000
Epoch 2/50
3/3 [==============================] - 0s 16ms/step - loss: 1.3406 - accuracy: 0.2708 - val_loss: 1.0394 - val_accuracy: 0.2917
Epoch 3/50
3/3 [==============================] - 0s 24ms/step - loss: 1.1470 - accuracy: 0.4375 - val_loss: 1.0107 - val_accuracy: 0.3750
Epoch 4/50
3/3 [==============================] - 0s 16ms/step - loss: 1.1683 - accuracy: 0.3542 - val_loss: 0.9870 - val_accuracy: 0.5000
Epoch 5/50
3/3 [==============================] - 0s 16ms/step - loss: 1.0564 - accuracy: 0.4375 - val_loss: 0.9448 - val_accuracy: 0.5000
Epoch 6/50
3/3 [==============================] - 0s 16ms/step - loss: 1.0334 - accuracy: 0.5104 - val_loss: 0.9082 - val_accuracy: 0.5000
Epoch 7/50
3/3 [==============================] - 0s 17ms/step - loss: 0.9332 - accuracy: 0.5104 - val_loss: 0.8659 - val_accuracy: 0.5000
Epoch 8/50
3/3 [==========

In [ ]:
y_pred_prob = model.predict(X_test)

y_pred = np.argmax(y_pred_prob, axis=1)

accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print(f'Test Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
print('\nConfusion Matrix:')
print(conf_matrix)

class_report = classification_report(np.argmax(y_test, axis=1), y_pred)
print('\nClassification Report:')
print(class_report)

roc_auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr')
print(f'\nROC-AUC Score: {roc_auc * 100:.2f}%')

1/1 [==============================] - 0s 17ms/step
Test Accuracy: 96.67%
Precision: 0.97
Recall: 0.97
F1 Score: 0.97

Confusion Matrix:
[[10  0  0]
 [ 0  8  1]
 [ 0  0 11]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.89      0.94         9
           2       0.92      1.00      0.96        11

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30


ROC-AUC Score: 97.00%


## Вопросы к заданию 2

1. Перечислите все гиперпараметры и их значения вот в таком формате (значения приведены просто для примера, у вас могут быть другие):

* количество эпох: 5
* скорость обучения: 0.0001
* функция активация на скрытых слоях: ReLU
* и т.д.

2. Сколько у вашей модели параметров?

Количество эпох: 50

Размер батча: 32

Скорость обучения (learning rate): 0.001 (задано в оптимизаторе Adam)

Функции активации на скрытых слоях:

Первый скрытый слой: ReLU
Второй скрытый слой: ReLU
Выходной слой: Softmax (для многоклассовой классификации)
Функция потерь: categorical_crossentropy (подходит для многоклассовой классификации)

Метрика для оценки качества модели: accuracy

Dropout:

Для первого скрытого слоя: 0.5
Для второго скрытого слоя: 0.3

Первый слой (Dense(128, input_dim=X_train.shape[1], activation='relu')):

Веса: 128 (нейроны) * 4 (входные признаки) = 512
Смещения: 128
Всего: 512 + 128 = 640 параметров
Второй слой (Dense(64, activation='relu')):

Веса: 128 (входные нейроны) * 64 (нейроны в слое) = 8192
Смещения: 64
Всего: 8192 + 64 = 8256 параметров
Третий слой (Dense(3, activation='softmax')):

Веса: 64 (входные нейроны) * 3 (нейроны в слое) = 192
Смещения: 3
Всего: 192 + 3 = 195 параметров


640 + 8256 + 195 = 9091 параметров

# Задание 3 (2 балла)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical

data = pd.merge(transactions, gender_train, on="customer_id", how="left")

data["gender"].fillna(-1, inplace=True)

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

X_train = train_data[["mcc_code", "amount", "term_id"]]
y_train = train_data["gender"]

X_test = test_data[["mcc_code", "amount", "term_id"]]
y_test = test_data["gender"]

scaler = MinMaxScaler()
X_train["amount"] = scaler.fit_transform(X_train[["amount"]])
X_test["amount"] = scaler.transform(X_test[["amount"]])

label_encoder = LabelEncoder()
unique_terms = set(X_train["term_id"].unique()).union(set(X_test["term_id"].unique()))
label_encoder.fit(list(unique_terms))

X_train["term_id"] = label_encoder.transform(X_train["term_id"])
X_test["term_id"] = label_encoder.transform(X_test["term_id"])

y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.3, verbose=1)

<ipython-input-2-ba69f5454d96>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["amount"] = scaler.fit_transform(X_train[["amount"]])
<ipython-input-2-ba69f5454d96>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["amount"] = scaler.transform(X_test[["amount"]])
<ipython-input-2-ba69f5454d96>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

Epoch 1/20
17500/17500 [==============================] - 81s 4ms/step - loss: 14.6395 - accuracy: 0.3971 - val_loss: 1.0843 - val_accuracy: 0.3998
Epoch 2/20
17500/17500 [==============================] - 79s 5ms/step - loss: 1.0910 - accuracy: 0.3990 - val_loss: 1.0843 - val_accuracy: 0.3998
Epoch 3/20
17500/17500 [==============================] - 77s 4ms/step - loss: 1.0926 - accuracy: 0.3990 - val_loss: 1.0842 - val_accuracy: 0.3998
Epoch 4/20
17500/17500 [==============================] - 83s 5ms/step - loss: 1.0919 - accuracy: 0.3990 - val_loss: 1.0843 - val_accuracy: 0.3998
Epoch 5/20
17500/17500 [==============================] - 77s 4ms/step - loss: 1.0870 - accuracy: 0.3990 - val_loss: 1.0843 - val_accuracy: 0.3998
Epoch 6/20
17500/17500 [==============================] - 80s 5ms/step - loss: 1.0889 - accuracy: 0.3990 - val_loss: 1.0843 - val_accuracy: 0.3998
Epoch 7/20
17500/17500 [==============================] - 78s 4ms/step - loss: 1.0924 - accuracy: 0.3990 - val_loss: 

KeyboardInterrupt: ignored

In [ ]:
# Оценка качества модели на тестовом наборе
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test_classes, y_pred)
precision = precision_score(y_test_classes, y_pred, average='weighted')
recall = recall_score(y_test_classes, y_pred, average='weighted')
f1 = f1_score(y_test_classes, y_pred, average='weighted')

conf_matrix = confusion_matrix(y_test_classes, y_pred)
class_report = classification_report(y_test_classes, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr')

# Вывод результатов
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'F1 Score: {f1 * 100:.2f}%')

print('\nConfusion Matrix:')
print(conf_matrix)

print('\nClassification Report:')
print(class_report)

print(f'\nROC-AUC Score: {roc_auc * 100:.2f}%')

6250/6250 [==============================] - 12s 2ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

Accuracy: 40.10%
Precision: 16.08%
Recall: 40.10%
F1 Score: 22.96%

Confusion Matrix:
[[    0     0 52555]
 [    0     0 67243]
 [    0     0 80202]]

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     52555
           1       0.00      0.00      0.00     67243
           2       0.40      1.00      0.57     80202

    accuracy                           0.40    200000
   macro avg       0.13      0.33      0.19    200000
weighted avg       0.16      0.40      0.23    200000


ROC-AUC Score: 50.00%


Оцените качество модели не менее, чем на трех разных метриках.

## Вопросы к заданию 3

1. Выведите значения метрик.
2. Как улучшить модель?
3. Для чего может понадобиться предсказание пола клиента по его транзакциям?

2.
1) Научиться обрабатывать данные...
2) Оптимизация параметров модели, таких как learning rate, количество слоев и нейронов в них, функции активации и дропауты.
3) Использование кросс-валидации для оценки модели на различных подмножествах данных.
4) Увеличение размера обучающей выборки или использование методов аугментации данных.

3. Например, банки могут использовать это для определения наиболее подходящих кредитных предложений для клиентов различного пола, а розничные компании могут использовать это для персонализации предложений и скидок.

После завершения контрольной работы дайте ссылку на colab в отдельном followup **в своей теме на Piazza**. Дайте разрешение на его просмотр.

**Не создавайте новую тему, не присылайте решение файлом - только ссылкой.**

Не забудьте запустить код в последней ячейке для контроля времени выполнения.

In [ ]:
current_datetime = datetime.now()
print(current_datetime)

2023-12-15 08:29:23.655088
